In [1]:
import duckdb

In [2]:
import pandas as pd
import numpy as np

In [ ]:
# q = """
# SELECT strftime('%Y', date) AS year, SUM(beef) AS beef_total
# FROM meat
# GROUP BY year;
# """
# print(pysqldf(q).head())

### sample data for the practise (this is a fake dataset)

- dt: transaction date, could be string or datetime, flexible with hive sql
- oid: string, order id, primary key
- uid: customer id
- num_items: number of items purchased by customers per order
- tt_gmv: total price of each order (not per item!)

In [4]:
orders = pd.read_csv('sample_transaction.csv',dtype=str).head(19)


In [5]:
orders['tt_gmv']=orders['tt_gmv'].astype(float)
orders['num_items']=orders['num_items'].astype(int)

In [9]:
print(duckdb.query("""
SELECT *
FROM orders
""").df().head(5)
)

           dt oid   uid  num_items  tt_gmv
0  2023-01-01   1  1000          1    20.0
1  2023-01-02   2  1001          2    56.0
2  2023-02-05   3  1002          3    34.0
3  2023-06-10   4  1003          4   100.0
4  2023-07-01   5  1000          2    23.0


### rank function
rank function is one of the most frequently used window functions and row_number is the most popular one
- row_number() over(partition by xxx order by xxx)
    - 1,2,3,4
- rank() over(partition by xxx order by xxx)
    - 1,2,2,4
- dense_rank() over(partition by xxx order by xxx)
    - 1,2,2,3

Q1: for every customer (uid), get the most expensive order they have purchase, need return dt, oid, num_items as well
- highest order: max ttm

In [13]:
q = """
with rank_tbl as (
select *, row_number() over(partition by uid order by tt_gmv desc) as rank
from orders
)
select *
from rank_tbl
where rank = 1
order by tt_gmv desc
"""

In [14]:
print(duckdb.query(q).df())

           dt oid   uid  num_items  tt_gmv  rank
0  2023-07-15   6  1001          3   100.0     1
1  2023-06-10   4  1003          4   100.0     1
2  2023-11-01  16  1000          1    78.0     1
3  2023-07-16   7  1002          1    55.0     1


### percentile function
percentile function is widely used to select top performance targets (countries, supermarkets, customers etc)
- ntile()
- percent_rank()

Q: return top 25% orders by tt_gmv

In [17]:
q = """
with perc_tbl as (
    select *, ntile(4) over(order by tt_gmv desc) as percentile
    from orders
)
select *
from perc_tbl
--where percentile = 1
"""

In [18]:
print(duckdb.query(q).df())

            dt oid   uid  num_items  tt_gmv  percentile
0   2023-06-10   4  1003          4   100.0           1
1   2023-07-15   6  1001          3   100.0           1
2   2023-12-18  19  1003          5    99.0           1
3   2023-11-01  16  1000          1    78.0           1
4   2023-09-28  13  1001          5    66.0           1
5   2023-01-02   2  1001          2    56.0           2
6   2023-07-16   7  1002          1    55.0           2
7   2023-09-10  11  1002          4    45.0           2
8   2023-10-16  15  1002          3    45.0           2
9   2023-12-05  18  1000          2    45.0           2
10  2023-02-05   3  1002          3    34.0           3
11  2023-07-01   5  1000          2    23.0           3
12  2023-09-01  10  1001          3    23.0           3
13  2023-10-01  14  1002          2    23.0           3
14  2023-11-05  17  1000          1    23.0           3
15  2023-01-01   1  1000          1    20.0           4
16  2023-08-15   9  1000          2    18.0     

In [19]:
q = """
with perc_tbl as (
    select *, ntile(4) over(order by tt_gmv desc) as percentile
    from orders
)
select *
from perc_tbl
where percentile = 1
"""

In [20]:
print(duckdb.query(q).df())


           dt oid   uid  num_items  tt_gmv  percentile
0  2023-06-10   4  1003          4   100.0           1
1  2023-07-15   6  1001          3   100.0           1
2  2023-12-18  19  1003          5    99.0           1
3  2023-11-01  16  1000          1    78.0           1
4  2023-09-28  13  1001          5    66.0           1


In [25]:
q = """
with perc_tile as (
    select *, percent_rank() over(order by tt_gmv desc) as percentile
    from orders
)
select *
from perc_tile
where percentile <= 0.25

"""

In [26]:
print(duckdb.query(q).df())


           dt oid   uid  num_items  tt_gmv  percentile
0  2023-06-10   4  1003          4   100.0    0.000000
1  2023-07-15   6  1001          3   100.0    0.000000
2  2023-12-18  19  1003          5    99.0    0.111111
3  2023-11-01  16  1000          1    78.0    0.166667
4  2023-09-28  13  1001          5    66.0    0.222222


In [27]:
q = """
with perc_tile as (
    select *, percent_rank() over(order by tt_gmv desc) as percentile
    from orders
)
select *
from perc_tile
--where percentile <= 0.25

"""

In [28]:
print(duckdb.query(q).df())


            dt oid   uid  num_items  tt_gmv  percentile
0   2023-06-10   4  1003          4   100.0    0.000000
1   2023-07-15   6  1001          3   100.0    0.000000
2   2023-12-18  19  1003          5    99.0    0.111111
3   2023-11-01  16  1000          1    78.0    0.166667
4   2023-09-28  13  1001          5    66.0    0.222222
5   2023-01-02   2  1001          2    56.0    0.277778
6   2023-07-16   7  1002          1    55.0    0.333333
7   2023-09-10  11  1002          4    45.0    0.388889
8   2023-10-16  15  1002          3    45.0    0.388889
9   2023-12-05  18  1000          2    45.0    0.388889
10  2023-02-05   3  1002          3    34.0    0.555556
11  2023-07-01   5  1000          2    23.0    0.611111
12  2023-09-01  10  1001          3    23.0    0.611111
13  2023-10-01  14  1002          2    23.0    0.611111
14  2023-11-05  17  1000          1    23.0    0.611111
15  2023-01-01   1  1000          1    20.0    0.833333
16  2023-08-15   9  1000          2    18.0    0

## lead lag function

- lead()
- lag()

Q2: use another data set for login detection, the goal is to get the user's most recent normal login frequency
- defition of nornal: consecutive is_abnormal = 0

In [29]:
login = pd.read_csv('login_detection.csv',dtype=str)
login

,timestamp,uid,is_abnormal
0,2023-11-01,123,0
1,2023-11-02,123,0
2,2023-11-03,123,1
3,2023-11-04,123,1
4,2023-11-05,123,1
5,2023-11-06,123,1
6,2023-11-07,123,1
7,2023-11-08,123,0
8,2023-11-09,123,1
9,2023-11-10,123,0


In [32]:
q = """
with lead_tbl as (
select *,lead(is_abnormal,1) over(partition by uid order by timestamp desc) as lead_normal,
row_number() over(partition by uid order by timestamp desc) rank
from login
)
select * from lead_tbl

"""

In [33]:
print(duckdb.query(q).df())


     timestamp  uid is_abnormal lead_normal  rank
0   2023-11-11  123           0           0     1
1   2023-11-10  123           0           1     2
2   2023-11-09  123           1           0     3
3   2023-11-08  123           0           1     4
4   2023-11-07  123           1           1     5
5   2023-11-06  123           1           1     6
6   2023-11-05  123           1           1     7
7   2023-11-04  123           1           1     8
8   2023-11-03  123           1           0     9
9   2023-11-02  123           0           0    10
10  2023-11-01  123           0        None    11


In [40]:
q = """
with lag_tbl as (
    select *,
    lag(is_abnormal,1) over(partition by uid order by timestamp) as lag_normal,
    rank() over(partition by uid order by timestamp desc) as rank
    from login
)
    select *
from lag_tbl

"""

In [41]:
print(duckdb.query(q).df())


     timestamp  uid is_abnormal lag_normal  rank
0   2023-11-11  123           0          0     1
1   2023-11-10  123           0          1     2
2   2023-11-09  123           1          0     3
3   2023-11-08  123           0          1     4
4   2023-11-07  123           1          1     5
5   2023-11-06  123           1          1     6
6   2023-11-05  123           1          1     7
7   2023-11-04  123           1          1     8
8   2023-11-03  123           1          0     9
9   2023-11-02  123           0          0    10
10  2023-11-01  123           0       None    11


## sum over

- calculating cumulative aggregation

In [42]:
gmv = pd.DataFrame({
    "uid": [1, 1, 1, 2, 2],
    "date": ["2024-01-01", "2024-01-02", "2024-01-03",
             "2024-01-01", "2024-01-02"],
    "gmv": [100, 200, 50, 300, 150]
})

In [44]:
q = """
select *,
sum(gmv) over(partition by uid order by date) as cum_sum
from gmv

"""

In [45]:
print(duckdb.query(q).df())


   uid        date  gmv  cum_sum
0    1  2024-01-01  100    100.0
1    1  2024-01-02  200    300.0
2    1  2024-01-03   50    350.0
3    2  2024-01-01  300    300.0
4    2  2024-01-02  150    450.0


In [46]:
q = """
select *,
sum(gmv) over(partition by uid order by date
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cum_sum
from gmv

"""

In [47]:
print(duckdb.query(q).df())


   uid        date  gmv  cum_sum
0    2  2024-01-01  300    300.0
1    2  2024-01-02  150    450.0
2    1  2024-01-01  100    100.0
3    1  2024-01-02  200    300.0
4    1  2024-01-03   50    350.0


In [49]:
q = """
select *,
sum(gmv) over(partition by uid) as sum_per_uvi
from gmv

"""
print(duckdb.query(q).df())

   uid        date  gmv  sum_per_uvi
0    1  2024-01-01  100        350.0
1    1  2024-01-02  200        350.0
2    1  2024-01-03   50        350.0
3    2  2024-01-01  300        450.0
4    2  2024-01-02  150        450.0
